In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC

from sklearn.metrics import accuracy_score,f1_score,classification_report

from sklearn.model_selection import GridSearchCV

In [ ]:
#데이터 읽기
train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/머신러닝/project/Train.csv')
test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/머신러닝/project/Test.csv')

In [ ]:
#index drop
train = train.drop('Index',axis=1)
test = test.drop('Index',axis=1)

In [ ]:
#데이터 분리
X_train, X_test, y_train, y_test = train_test_split(train.loc[:,'X0':'X999'], train.iloc[:,-1],test_size=0.2,random_state=100)

In [ ]:
#svm parameter tuning
param_dist = {
    'C' : [0.001, 0.01, 0.1, 1, 10, 100],
    'gamma' : [0.001, 0.01, 0.1, 1, 10, 100]
}

gs = GridSearchCV(SVC(), param_dist, cv=5,scoring='f1_macro')
gs.fit(X_train,y_train)

print('test f1 : ', gs.score(X_test, y_test))
print('최적의 parameters : ', gs.best_params_)

test f1 :  0.693110971913348
최적의 parameters :  {'C': 10, 'gamma': 0.01}


In [ ]:
def result(X_train, X_test, y_train, y_test):
    m1=SVC(kernel="rbf",C=10, gamma=0.01)
    m1.fit(X_train,y_train)
    y_pred = m1.predict(X_test)
    f1 = f1_score(y_test,y_pred,average='binary',pos_label='positive')
    return f1

In [ ]:
#f1-score가 잘 나오는 randomstate 찾기
from imblearn.under_sampling import *
max=0
best=0
for i in range(0,101,1):
  X_train, X_test, y_train, y_test = train_test_split(train.loc[:,'X0':'X999'], train.iloc[:,-1],test_size=0.2,random_state=i)
  X,y = EditedNearestNeighbours(n_neighbors=3).fit_resample(X_train,y_train)
  a=result(X, X_test, y, y_test)
  if(max<a):
    max=a
    best=i
  print('현재 i: ', i)
  print('최고의 random_state: ',best)
  print('f1_score: ', max)

현재 i:  0
최고의 random_state:  0
f1_score:  0.5119047619047619
현재 i:  1
최고의 random_state:  0
f1_score:  0.5119047619047619
현재 i:  2
최고의 random_state:  2
f1_score:  0.5656565656565657
현재 i:  3
최고의 random_state:  2
f1_score:  0.5656565656565657
현재 i:  4
최고의 random_state:  4
f1_score:  0.6000000000000001
현재 i:  5
최고의 random_state:  4
f1_score:  0.6000000000000001
현재 i:  6
최고의 random_state:  4
f1_score:  0.6000000000000001
현재 i:  7
최고의 random_state:  4
f1_score:  0.6000000000000001
현재 i:  8
최고의 random_state:  4
f1_score:  0.6000000000000001
현재 i:  9
최고의 random_state:  4
f1_score:  0.6000000000000001
현재 i:  10
최고의 random_state:  4
f1_score:  0.6000000000000001
현재 i:  11
최고의 random_state:  4
f1_score:  0.6000000000000001
현재 i:  12
최고의 random_state:  4
f1_score:  0.6000000000000001
현재 i:  13
최고의 random_state:  4
f1_score:  0.6000000000000001
현재 i:  14
최고의 random_state:  4
f1_score:  0.6000000000000001
현재 i:  15
최고의 random_state:  4
f1_score:  0.6000000000000001
현재 i:  16
최고의 random_state:  4
f1_

In [ ]:
#Best randome state로 데이터 분리
X_train, X_test, y_train, y_test = train_test_split(train.loc[:,'X0':'X999'], train.iloc[:,-1],test_size=0.2,random_state=30)

In [ ]:
reX_train,rey_train = EditedNearestNeighbours(n_neighbors=3).fit_resample(X_train,y_train)

In [ ]:
m=SVC(kernel='rbf',C=10, gamma=0.01)
m.fit(reX_train,rey_train)

pred_train = m.predict(reX_train)
pred_test = m.predict(X_test)

print("Classificcation Report: \n{}".format(classification_report(y_test, pred_test)))

Classificcation Report: 
              precision    recall  f1-score   support

    negative       0.97      0.97      0.97       919
    positive       0.69      0.63      0.66        81

    accuracy                           0.95      1000
   macro avg       0.83      0.80      0.81      1000
weighted avg       0.95      0.95      0.95      1000



In [ ]:
test_X=test.loc[:,'X0':'X999']
prediction = m.predict(test_X)

print(f'Type : {type(prediction)} | Length : {len(prediction)}')

Type : <class 'numpy.ndarray'> | Length : 3500


In [ ]:
np.unique(prediction, return_counts=True)

(array(['negative', 'positive'], dtype=object), array([3216,  284]))

In [ ]:
import numpy as np

np.savetxt('/content/drive/MyDrive/Colab Notebooks/머신러닝/project/박정원.csv', prediction, delimiter=',',fmt='%s')